# Extract 2D obstacles
Generate polygons for all static obstacles, and merge these with the sidewalk polygons.

In [ ]:
# Add project src to path.
import set_path

import numpy as np
import pandas as pd
import geopandas as gpd
import os
import pathlib
import pickle
import shapely.geometry as sg
import shapely.ops as so
from tqdm.notebook import tqdm
tqdm.pandas()

from upcp.utils import bgt_utils
from upcp.utils import csv_utils
from upcp.utils import las_utils

from upc_sw.cluster2polygon import Cluster2Polygon
from upc_sw import sw_utils
from upc_sw import poly_utils

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

## Create polygons of static obstacles
In the previous notebook, we performed a change detection algorithm that calculated M3C2 distance for each point in the point cloud. Based on negative and positive threshold values we can filter for the static points in the point cloud. We then cluster these into individual obstacles and create bounding polygons for each.

In [ ]:
### SETTINGS ###

CRS = 'epsg:28992'

base_folder = '../../datasets/Accessibility_oost/'
pc_data_folder = f'{base_folder}pointclouds/'
bgt_folder = f'{base_folder}bgt/'
out_folder = f'{base_folder}output/'

# Scraped sidewalk and terras data for the area (see Notebook 1c)
sidewalk_data = f'{bgt_folder}bgt_voetpad.gpkg'
terras_data = f'{bgt_folder}terras_data.gpkg'
obstacle_data = f'{bgt_folder}obstacle_data.gpkg'

# Output file
output_file = f'{out_folder}obstacles.gpkg'

# Allow resume by saving intermediate output
resume = True
resume_batch_size = 10
tmp_file = f'{out_folder}obst_tmp.pkl'

# Distance threshold for static obstacles.
m3c2_threshold = 0.2

# Convert 3D Obstacle blobs to 2D polygons using a clustering algorithm.
# Set use_concave=False to use the faster convex hull.
# Change alpha to determine the 'concaveness' of the concave hull, with 0 being convex.
c2p = Cluster2Polygon(min_component_size=100, grid_size=0.2, use_concave=True, concave_min_area=2.5, alpha=0.5)

# Create output folder if it doesn't exist
pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
all_tiles = las_utils.get_tilecodes_from_folder(f'{pc_data_folder}m3c2/')

In [ ]:
if resume and os.path.exists(tmp_file):
    with open(tmp_file, 'rb') as f:
        static_obstacles = pickle.load(f)
        all_tiles = all_tiles - set(static_obstacles['tilecode'])
else:
    static_obstacles = {'tilecode': [],
                        'type': [],
                        'geometry': []}

In [ ]:
len(all_tiles)

In [ ]:
tile_tqdm = tqdm(all_tiles, unit='tile', smoothing=0)

for i, tilecode in enumerate(tile_tqdm):
    tile_tqdm.set_postfix_str(tilecode)
    
    # Read point cloud with M3C2 distances
    in_file = f'{pc_data_folder}m3c2/m3c2_{tilecode}.laz'
    points, m3c2_distance = sw_utils.read_las(in_file, extra_val='M3C2_distance', extra_val_dtype='float32')

    # Filter for static points
    mask = np.abs(m3c2_distance) < m3c2_threshold
    
    if np.count_nonzero(mask) > 0:
        # Get the polygons
        polygons, types = c2p.get_obstacle_polygons(points[mask])
        static_obstacles['tilecode'].extend([tilecode]*len(polygons))
        static_obstacles['type'].extend(types)
        static_obstacles['geometry'].extend(polygons)
    
    if i % resume_batch_size == 0:
        with open(tmp_file, 'wb') as f:
            pickle.dump(static_obstacles, f)

static_obstacles_gdf = gpd.GeoDataFrame(static_obstacles, geometry='geometry', crs=CRS)

In [ ]:
# Fix invalid polygons
static_obstacles_gdf['geometry'] = static_obstacles_gdf['geometry'].progress_apply(poly_utils.fix_invalid)

In [ ]:
# Save the obstacle GeoDataFrame.
if len(static_obstacles_gdf) > 0:
    static_obstacles_gdf.to_file(output_file, driver='GPKG')
else:
    print('No obstacle data to write.')

## Merge sidewalk polygons with obstacles
The found obstacles (polygons) in the previous step are merged with the sidewalk polygons as interiors.

In [ ]:
### SETTINGS ###

merged_output_file = f'{out_folder}sidewalks_with_obstacles.gpkg'

# Buffer width around the sidewalk to preserve its shape. Set to '0' to ignore this.
sw_buffer = 0.01

# Add padding around obstacles.
obstacle_padding = 0.05

In [ ]:
# Load the sidewalk data
sidewalk_gdf = gpd.read_file(sidewalk_data, crs=CRS).set_index('ogc_fid')

# Load the "terras" data
if pathlib.Path(terras_data).is_file():
    terras_gdf = gpd.read_file(terras_data, crs=CRS).set_index('id')
else:
    terras_gdf = gpd.GeoDataFrame({'geometry': []}, geometry='geometry', crs=CRS)

# Load the obstacle data
static_obstacles_gdf = gpd.read_file(output_file, crs=CRS)

In [ ]:
def merge_obstacles(row):
    # Subtract all obstacles that intersect the sidewalk polygon.
    sw_ext, sw_int = poly_utils.extract_interior(row.geometry)
    obst_poly = (static_obstacles_gdf[static_obstacles_gdf.intersects(sw_ext)]
                 .buffer(obstacle_padding)
                 .unary_union)
    # TODO buffer also for terrace shapes?
    terras_poly = terras_gdf[terras_gdf.intersects(sw_ext)].unary_union
    merged_poly = so.unary_union([poly for poly in [obst_poly, terras_poly, sw_int] if poly is not None])
    # TODO: do something with obstacle type?
    if merged_poly is not None:
        sw_ext = sw_ext.buffer(sw_buffer) - sw_ext.intersection(merged_poly)

    return sw_ext

sw_merged_gdf = sidewalk_gdf.copy()
sw_merged_gdf['geometry'] = sidewalk_gdf.progress_apply(merge_obstacles, axis=1)

In [ ]:
# Save the merged sidewalk data.
sw_merged_gdf.to_file(merged_output_file, driver='GPKG')